In [2]:
# Import necessary libraries
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv("../../data/job/rallit_text.csv", quoting=2)
print(df.head())

# Map job labels to numbers and rename to 'labels' for compatibility with Hugging Face
job_map = {
    "PM": 0, "Sales": 1, "데브옵스 엔지니어": 2, "데이터 분석가": 3,
    "데이터 엔지니어": 4, "백엔드 개발자": 5, "풀스택 개발자": 6, "프론트엔드 개발자": 7
}
df['labels'] = df['job'].map(job_map)
df.dropna(inplace=True)

# Clean text data
def clean_text(text):
    # Add your cleaning rules here
    text = re.sub('[^가-힣a-zA-Z0-9]', ' ', text)
    return text

df['text'] = df['text'].apply(clean_text)

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=len(job_map))

# Prepare the datasets
def tokenize_data(example):
    model_inputs = tokenizer(example['text'], max_length=128, padding="max_length", truncation=True)
    model_inputs['labels'] = example['labels']
    return model_inputs

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='models/kor_base', #모델 저장
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

# Define a metrics function for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro')
    _, _, f1_weighted, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    _, _, f1_micro, _ = precision_recall_fscore_support(labels, preds, average='micro')
    accuracy = accuracy_score(labels, preds)
    return {
        "accuracy": accuracy,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
        "f1_micro": f1_micro
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train and evaluate the model
trainer.train()
trainer.evaluate()

         job                                               text
0    백엔드 개발자  기술 스택: Python, Java, C, Django, Spring Boot, F...
1  프론트엔드 개발자  기술 스택: JavaScript, TypeScript, React, Python, ...
2    백엔드 개발자  기술 스택: Java\r\nSpring Boot\r\nJPA\r\nTypeScrip...
3    백엔드 개발자  기술 스택: Java\r\nKotlin\r\nJavaScript\r\nTypeScr...
4    풀스택 개발자  기술 스택: JavaScript, Docker, Elasticsearch, Vue....


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/564 [00:00<?, ? examples/s]

Map:   0%|          | 0/141 [00:00<?, ? examples/s]

  0%|          | 0/720 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.4733200073242188, 'eval_accuracy': 0.7801418439716312, 'eval_f1_macro': 0.4334761385470219, 'eval_f1_weighted': 0.7658615435779639, 'eval_f1_micro': 0.7801418439716312, 'eval_runtime': 2.4546, 'eval_samples_per_second': 57.443, 'eval_steps_per_second': 1.222, 'epoch': 19.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.4708234071731567, 'eval_accuracy': 0.7801418439716312, 'eval_f1_macro': 0.4778756957328386, 'eval_f1_weighted': 0.7589941972920696, 'eval_f1_micro': 0.7801418439716312, 'eval_runtime': 2.4562, 'eval_samples_per_second': 57.405, 'eval_steps_per_second': 1.221, 'epoch': 20.0}
{'train_runtime': 674.5137, 'train_samples_per_second': 16.723, 'train_steps_per_second': 1.067, 'train_loss': 0.41724361843532987, 'epoch': 20.0}


  0%|          | 0/3 [00:00<?, ?it/s]